In [1]:
import torch
import random 
import numpy as np 
from tqdm import tqdm 
from scipy.spatial.distance import cdist, cosine
from scipy.optimize import linear_sum_assignment
from utils.corrupt_graph import remove_edge, remove_node, add_edge, add_node
from utils.query_machine import get_candidates
from python_emb import *
import torch.nn.functional as F
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr, spearmanr
from collections import defaultdict

In [ ]:
for dataset in ['yeast', 'human', 'cora', 'citeseer', 'pubmed', 'wordnet']:
    dim=4
    ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

    ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], dim, ori_graph_data.num_class)
    ori_emb_model = ori_emb_model.cuda()
    ori_emb_model.load_state_dict(torch.load('model_dim/{}_sup_20_dim{}.pt'.format(dataset, dim)))
    ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
    ori_emb_model.eval()

    ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
    ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


    for i, node in enumerate(ori_graph_data.G.nodes):
        ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

    query_machine = GraphQuery(ori_emb_model, 
            ori_graph_emb,
            ori_graph_data.G,
            ori_graph_data.id_map,
            ori_graph_data.feats,
            ori_graph_data.raw_feats,
            ori_graph_data.full_adj,
            ori_graph_data.deg)

    def get_embedding_subgraph(subgraph):
        sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
        embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
        embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

        return embedding_subgraph, sub_id_map


    mcs2emb = defaultdict(list)
    GRAPH_SIZE = 15
    graphs = []
    for i, core_node in tqdm(enumerate(query_machine.ori_graph.nodes)):
        biggraph = query_machine.create_subgraph_from_core(core_node, GRAPH_SIZE)
        if len(biggraph) != GRAPH_SIZE:
            continue
        sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

        embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
        embedding_biggraph = embedding_biggraph.detach().cpu().numpy()
        subgraphs = {}
        for dist in range(1,8):
            connected = False
            n_tries  = 0
            while(not connected and n_tries<=10):
                subgraph = remove_edge(biggraph, dist)
                n_tries += 1
                if len(subgraph)==0: continue
                connected = nx.is_connected(subgraph)
                if (n_tries>10): break
            if n_tries <=10 and nx.is_connected(subgraph):
                sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
                embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
                embedding_subgraph = embedding_subgraph.detach().cpu().numpy()
                emb_sim = ((embedding_biggraph.mean(0)- embedding_subgraph.mean(0))**2).sum()
                if not np.isnan(emb_sim):
                    mcs2emb[dist].append(emb_sim)
                    subgraphs[dist] = subgraph
        graphs.append([biggraph, subgraphs])
    X = []
    Y = []
    for k,v in mcs2emb.items():
        X.append(k)
        Y.append(sum(v)/len(v))
    print(dataset, dim, spearmanr(X, Y), pearsonr(X,Y))
        
    for dim in [6, 8, 10, 12, 14, 16]:
        ori_graph_data = load_data('./dataspace/graph/{}/{}'.format(dataset, dataset), supervised=False, max_degree=5, multiclass=False, use_random_walks=False)

        ori_emb_model = SupervisedGraphSage(ori_graph_data.raw_feats.shape[1], dim, ori_graph_data.num_class)
        ori_emb_model = ori_emb_model.cuda()
        ori_emb_model.load_state_dict(torch.load('model_dim/{}_sup_20_dim{}.pt'.format(dataset, dim)))
        ori_emb_model.set_params(ori_graph_data.full_adj, ori_graph_data.deg, ori_graph_data.feats)
        ori_emb_model.eval()

        ori_graph_emb = F.normalize(ori_emb_model.aggregator(list(range(ori_graph_data.raw_feats.shape[0]))), dim = 1)
        ori_graph_emb = ori_graph_emb.detach().cpu().numpy()


        for i, node in enumerate(ori_graph_data.G.nodes):
            ori_graph_data.G.nodes[node]['label'] = ori_graph_data.multi2single_label[tuple(ori_graph_data.G.nodes[node]['label'])]

        query_machine = GraphQuery(ori_emb_model, 
                ori_graph_emb,
                ori_graph_data.G,
                ori_graph_data.id_map,
                ori_graph_data.feats,
                ori_graph_data.raw_feats,
                ori_graph_data.full_adj,
                ori_graph_data.deg)

        def get_embedding_subgraph(subgraph):
            sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
            embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
            embedding_subgraph = embedding_subgraph.detach().cpu().numpy()

            return embedding_subgraph, sub_id_map


        mcs2emb = defaultdict(list)

        for biggraph, subgraphs in tqdm(graphs):
            sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(biggraph)

            embedding_biggraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
            embedding_biggraph = embedding_biggraph.detach().cpu().numpy()
            for dist, subgraph in subgraphs.items():                
                sub_id_map, sub_raw_feats, all_sub_adj, sub_degree = query_machine.create_subgraph_map(subgraph)
                embedding_subgraph = query_machine.embedding_subgraph(sub_raw_feats, all_sub_adj, sub_degree)
                embedding_subgraph = embedding_subgraph.detach().cpu().numpy()
                emb_sim = ((embedding_biggraph.mean(0)- embedding_subgraph.mean(0))**2).sum()
                if not np.isnan(emb_sim):
                    mcs2emb[dist].append(emb_sim)
        X = []
        Y = []
        for k,v in mcs2emb.items():
            X.append(k)
            Y.append(sum(v)/len(v))
        print(dataset, dim, spearmanr(X, Y), pearsonr(X,Y))


Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/yeast/yeast-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/yeast/yeast-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/yeast/yeast-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 12519
Preprocessing finished, graph info:
Name: yeast
Type: Graph
Number of nodes: 3101
Number of edges: 12519
Average degree:   8.0742



0it [00:00, ?it/s]
4it [00:00, 38.15it/s]
8it [00:00, 37.22it/s]
13it [00:00, 38.94it/s]
18it [00:00, 41.14it/s]
23it [00:00, 42.63it/s]
28it [00:00, 42.84it/s]
33it [00:00, 43.20it/s]
38it [00:00, 43.87it/s]
43it [00:00, 44.98it/s]
48it [00:01, 45.17it/s]
53it [00:01, 44.18it/s]
58it [00:01, 42.83it/s]
63it [00:01, 42.78it/s]
68it [00:01, 43.67it/s]
73it [00:01, 45.11it/s]
78it [00:01, 43.79it/s]
83it [00:01, 45.06it/s]
88it [00:02, 44.86it/s]
93it [00:02, 44.47it/s]
98it [00:02, 43.40it/s]
103it [00:02, 43.83it/s]
108it [00:02, 44.44it/s]
113it [00:02, 43.70it/s]
118it [00:02, 44.46it/s]
123it [00:02, 43.91it/s]
128it [00:02, 45.48it/s]
133it [00:03, 46.20it/s]
138it [00:03, 47.08it/s]
143it [00:03, 47.49it/s]
148it [00:03, 47.40it/s]
154it [00:03, 48.57it/s]
159it [00:03, 45.19it/s]
164it [00:03, 45.26it/s]
169it [00:03, 43.99it/s]
174it [00:03, 45.57it/s]
179it [00:04, 43.92it/s]
184it [00:04, 43.83it/s]
189it [00:04, 45.43it/s]
194it [00:04, 44.76it/s]
199it [00:04, 44.48it/s]
20

1545it [00:32, 47.67it/s]
1550it [00:32, 46.38it/s]
1555it [00:32, 45.57it/s]
1560it [00:33, 46.45it/s]
1565it [00:33, 46.12it/s]
1570it [00:33, 46.72it/s]
1575it [00:33, 47.30it/s]
1580it [00:33, 47.25it/s]
1587it [00:33, 50.98it/s]
1593it [00:33, 49.87it/s]
1599it [00:33, 50.06it/s]
1605it [00:33, 50.10it/s]
1611it [00:34, 48.93it/s]
1616it [00:34, 49.17it/s]
1621it [00:34, 48.32it/s]
1628it [00:34, 52.56it/s]
1634it [00:34, 50.51it/s]
1640it [00:34, 49.96it/s]
1646it [00:34, 48.43it/s]
1653it [00:34, 51.97it/s]
1659it [00:35, 50.63it/s]
1665it [00:35, 48.80it/s]
1670it [00:35, 48.52it/s]
1675it [00:35, 47.69it/s]
1680it [00:35, 47.97it/s]
1685it [00:35, 47.86it/s]
1692it [00:35, 52.21it/s]
1698it [00:35, 50.46it/s]
1704it [00:35, 51.87it/s]
1710it [00:36, 51.26it/s]
1716it [00:36, 49.69it/s]
1722it [00:36, 47.77it/s]
1730it [00:36, 52.31it/s]
1736it [00:36, 49.78it/s]
1742it [00:36, 50.31it/s]
1748it [00:36, 50.13it/s]
1754it [00:37, 39.37it/s]
1760it [00:37, 42.50it/s]
1765it [00:3

yeast 4 SpearmanrResult(correlation=1.0, pvalue=0.0) (0.9853032754730798, 4.989733096677769e-05)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/yeast/yeast-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/yeast/yeast-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/yeast/yeast-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 12519
Preprocessing finished, graph info:
Name: yeast
Type: Graph
Number of nodes: 3101
Number of edges: 12519
Average degree:   8.0742



 69%|██████▉   | 2051/2972 [00:16<00:07, 123.42it/s]


...

100%|██████████| 74031/74031 [04:15<00:00, 289.39it/s]


wordnet 12 SpearmanrResult(correlation=-0.3571428571428572, pvalue=0.4316113520383278) (-0.2795634517229306, 0.5437252649688282)
Set max degree to 5
-----------------------------------------------
Loading data:
Loading graph data from ./dataspace/graph/wordnet/wordnet-G.json
Removed 0 nodes that lacked proper annotations due to networkx versioning issues
File loaded successfully
Loading feature from ./dataspace/graph/wordnet/wordnet-G.json
File loaded successfully
Loading classmap data from ./dataspace/graph/wordnet/wordnet-class_map.json
File loaded successfully
Loaded data.. now preprocessing..
Use original edges
Generate train edges
Number of training edges: 127124
Preprocessing finished, graph info:
Name: wordnet
Type: Graph
Number of nodes: 82670
Number of edges: 127124
Average degree:   3.0755



  7%|▋         | 5479/74031 [00:16<04:14, 269.20it/s]


 11%|█         | 8197/74031 [00:32<06:37, 165.77it/s]


 11%|█▏        | 8397/74031 [00:32<05:24, 202.02it/s]